In [1]:
# Libraries import
import cv2
import numpy as np

In [2]:
print(cv2.__version__)

4.10.0


In [3]:
# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

In [4]:
# Load the COCO class labels YOLO was trained on
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
# Define colors for each class label
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [6]:
# Load the video
cap = cv2.VideoCapture("1900-151662242_small.mp4")
# Video by <a href="https://pixabay.com/users/variousphotography-1860391/?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=1900">VariousPhotography</a> from <a href="https://pixabay.com//?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=1900">Pixabay</a>

In [7]:
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame, or end of video.")
        break
    
    height, width, channels = frame.shape

    # Prepare the frame for YOLO (resize, convert to blob)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Forward pass: Get the output layers for the frame
    outs = net.forward(output_layers)

    # Initialize lists for detected objects
    class_ids = []
    confidences = []
    boxes = []

    # Process the network outputs
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Coordinates for the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression to remove duplicate boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]

            # Draw the bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            # Add label and confidence score
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

    # Display the frame with detected objects
    cv2.imshow("Video", frame)

    # Break the loop if the Escape key is pressed
    if cv2.waitKey(1) & 0xFF == 27: 
        break


In [8]:
# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()